In [24]:
import torch
from torch import nn
from evotorch import Problem
from evotorch.algorithms import SNES
from evotorch.logging import StdOutLogger

from evotorch import Problem
from evotorch.algorithms import CEM
from evotorch.decorators import vectorized

from evotorch import Problem
from evotorch.algorithms import GeneticAlgorithm
from evotorch.decorators import vectorized
from evotorch.operators import SimulatedBinaryCrossOver
from evotorch.operators import PolynomialMutation

from evotorch import Problem
from evotorch.algorithms import GeneticAlgorithm
from evotorch.decorators import vectorized
from evotorch.operators import TwoPointCrossOver



from evotorch.algorithms import XNES


In [25]:
# #prompt for output
def prompt(parameters: torch.Tensor) -> torch.Tensor:
    #prompt user for input from jupyter notebook
    print('Please run instrument with voltages {}'.format(parameters))
    return torch.tensor(float(input("Enter a number: ")))
# @vectorized
# def fitness(x: torch.Tensor) -> torch.Tensor:
#     return torch.linalg.norm(x, dim=-1)

In [37]:
problem = Problem(
    'max',
    prompt,
   # fitness,
    solution_length = 3,
    #initial_bounds=([35,50],[45,100]),
    bounds=(
    [30,1,0.33],
    [140,45,0.66]
            ),
    dtype=torch.float32,
    #device='cpu'
)
problem
# components = {
#               'temperature':np.arange(30,140,3).tolist(),   # Discrete grid of concentrations
#               'time': np.arange(1,45,2).tolist(),
#               'mfp': np.arange(0.33,0.66,0.025).tolist()}

[2023-03-04 04:36:35] INFO     < 3850> evotorch.core: Instance of `Problem` (id:140201642763216) -- The `dtype` for the problem's decision variables is set as torch.float32
[2023-03-04 04:36:35] INFO     < 3850> evotorch.core: Instance of `Problem` (id:140201642763216) -- `eval_dtype` (the dtype of the fitnesses and evaluation data) is set as torch.float32
[2023-03-04 04:36:35] INFO     < 3850> evotorch.core: Instance of `Problem` (id:140201642763216) -- The `device` of the problem is set as cpu
[2023-03-04 04:36:35] INFO     < 3850> evotorch.core: Instance of `Problem` (id:140201642763216) -- The number of actors that will be allocated for parallelized evaluation is 0


In [38]:
def my_own_gaussian_mutation(x: torch.Tensor) -> torch.Tensor:
    # The default GaussianMutation of EvoTorch does not (yet) support different standard deviation values
    # per variable. However, we can define our own mutation operator which adds noise of different magnitudes
    # to different variables, like in this example:
    [_, solution_length] = x.shape
    dtype = x.dtype
    device = x.device

    # Generate Gaussian noise where each column has its own magnitude
    noise = (
        torch.randn(solution_length, dtype=dtype, device=device)
        * torch.tensor([3, 2, 0.025], dtype=dtype, device=device)
    )

    return x + noise

In [44]:
#searcher = GeneticAlgorithm(problem, 
#                problem,
#                stdev_init=[1,5,2]
#                            )

searcher = GeneticAlgorithm(
    problem,
    popsize=1,
    operators=[my_own_gaussian_mutation]
       #TwoPointCrossOver(problem, tournament_size=1)
   # ],
)

_ = StdOutLogger(searcher, interval=1)

searcher.run(num_generations=1)
#searcher

Please run instrument with voltages ReadOnlyTensor([105.2008,  44.2528,   0.4365])
Please run instrument with voltages ReadOnlyTensor([102.5616,  41.0772,   0.4454])
         iter : 1
pop_best_eval : 5.0
    mean_eval : 5.0
  median_eval : 5.0
    best_eval : 87310.0
   worst_eval : 1.0



In [40]:
list(searcher.population[:10])

[<Solution values=tensor([76.6061, 26.8013,  0.3939]), evals=tensor([87310.])>,
 <Solution values=tensor([116.5776,   8.6780,   0.4550]), evals=tensor([63546.])>,
 <Solution values=tensor([115.4003,  33.1872,   0.3420]), evals=tensor([15661.])>,
 <Solution values=tensor([142.8780,  35.0637,   0.3597]), evals=tensor([6544.])>,
 <Solution values=tensor([52.8484, 28.9111,  0.6436]), evals=tensor([4851.])>]